## Tensorboard 高阶
使用Tensorboard的高阶可视化(weights,biases...)  
该示例使用了MNIST手写数据集
(http://yann.lecun.com/exdb/mnist/).

- Author: Aymeric Damien
- Project: https://github.com/aymericdamien/TensorFlow-Examples/

#### 1. 导入MNIST数据集

In [1]:
from __future__ import print_function

import tensorflow as tf

# Import MINST data
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("data/", one_hot=True)

Extracting data/train-images-idx3-ubyte.gz
Extracting data/train-labels-idx1-ubyte.gz
Extracting data/t10k-images-idx3-ubyte.gz
Extracting data/t10k-labels-idx1-ubyte.gz


#### 2. 自定义参数

In [2]:
#超参数
learning_rate = 0.01
training_epochs = 25
batch_size = 100
display_epoch = 1
logs_path = 'events/'   #当前目录下创建一个events文件夹以保存Tensorboard文件

#TensorFlow Graph输入
#MNIST数据的形状:28 * 28 = 784
x = tf.placeholder(tf.float32, [None, 784], name='InputData')
#0-9个数字作为类别标签
y = tf.placeholder(tf.float32, [None, 10], name='LabelData')

#设置模型参数
n_hidden_1 = 256   #第一隐层的特征数
n_hidden_2 = 256   #第二隐层的特征数
n_input = 784      #MNIST输入（图像形状：28*28像素）
n_classes = 10     #MNIST总的类别数（0-9个数字）
'''
# Parameters
learning_rate = 0.01
training_epochs = 25
batch_size = 100
display_step = 1
logs_path = 'events/'

# Network Parameters
n_hidden_1 = 256 # 1st layer number of features
n_hidden_2 = 256 # 2nd layer number of features
n_input = 784 # MNIST data input (img shape: 28*28)
n_classes = 10 # MNIST total classes (0-9 digits)

# tf Graph Input
# mnist data image of shape 28*28=784
x = tf.placeholder(tf.float32, [None, 784], name='InputData')
# 0-9 digits recognition => 10 classes
y = tf.placeholder(tf.float32, [None, 10], name='LabelData')
'''

"\n# Parameters\nlearning_rate = 0.01\ntraining_epochs = 25\nbatch_size = 100\ndisplay_step = 1\nlogs_path = 'events/'\n\n# Network Parameters\nn_hidden_1 = 256 # 1st layer number of features\nn_hidden_2 = 256 # 2nd layer number of features\nn_input = 784 # MNIST data input (img shape: 28*28)\nn_classes = 10 # MNIST total classes (0-9 digits)\n\n# tf Graph Input\n# mnist data image of shape 28*28=784\nx = tf.placeholder(tf.float32, [None, 784], name='InputData')\n# 0-9 digits recognition => 10 classes\ny = tf.placeholder(tf.float32, [None, 10], name='LabelData')\n"

#### 3. 构建模型

In [3]:
def multilayer_perceptron(x, weights, biases):
    #隐层，使用RELU作为激活函数
    layer_1 = tf.add(tf.matmul(x, weights['w1']), biases['b1'])
    layer_1 = tf.nn.relu(layer_1)
    #构建一个summary来可视化第一层的RELU激活函数
    tf.summary.histogram("relu1", layer_1)
    #隐层，使用RELU作为激活函数
    layer_2 = tf.add(tf.matmul(layer_1, weights['w2']), biases['b2'])
    layer_2 = tf.nn.relu(layer_2)
    #构建一个summary来可视化第二层的RELU激活函数
    tf.summary.histogram("relu2", layer_2)
    #输出层
    out_layer = tf.add(tf.matmul(layer_2, weights['w3']), biases['b3'])
    return out_layer

# 保存每层的权重
weights = {
    'w1': tf.Variable(tf.random_normal([n_input, n_hidden_1]), name='W1'),
    'w2': tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2]), name='W2'),
    'w3': tf.Variable(tf.random_normal([n_hidden_2, n_classes]), name='W3')
}
biases = {
    'b1': tf.Variable(tf.random_normal([n_hidden_1]), name='b1'),
    'b2': tf.Variable(tf.random_normal([n_hidden_2]), name='b2'),
    'b3': tf.Variable(tf.random_normal([n_classes]), name='b3')
}

#### 4. 将所有的operation封装到一个scope中

In [4]:
with tf.name_scope('Model'):
    #建立模型
    pred = multilayer_perceptron(x,weights,biases)

with tf.name_scope('Loss'):
    #softmax交叉熵
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=y,logits=pred)) 
with tf.name_scope('SGD'):
    #梯度下降
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    
    #计算每个变量的梯度
    grads = tf.gradients(loss,tf.trainable_variables())
    grads = list(zip(grads,tf.trainable_variables()))
    #通过梯度更新变量
    apply_grads = optimizer.apply_gradients(grads_and_vars = grads)
    
with tf.name_scope('Accuracy'):
    acc = tf.equal(tf.argmax(pred,1),tf.argmax(y,1))
    acc = tf.reduce_mean(tf.cast(acc,tf.float32))
    

In [5]:
# 初始化变量
init = tf.global_variables_initializer()

#构建cost的summary
tf.summary.scalar('loss',loss)
#构建accuracy的summary
tf.summary.scalar('accuracy',acc)
#构建可视化weights的summary
for var in tf.trainable_variables():
    tf.summary.histogram(var.name,var)
#SUMMARIZE所有的梯度
for grad,var in grads:
    tf.summary.histogram(var.name + '/gradient',grad)
#合并所有summary到一个operation
merged_summary_op = tf.summary.merge_all()



INFO:tensorflow:Summary name W1:0 is illegal; using W1_0 instead.
INFO:tensorflow:Summary name W2:0 is illegal; using W2_0 instead.
INFO:tensorflow:Summary name W3:0 is illegal; using W3_0 instead.
INFO:tensorflow:Summary name b1:0 is illegal; using b1_0 instead.
INFO:tensorflow:Summary name b2:0 is illegal; using b2_0 instead.
INFO:tensorflow:Summary name b3:0 is illegal; using b3_0 instead.
INFO:tensorflow:Summary name W1:0/gradient is illegal; using W1_0/gradient instead.
INFO:tensorflow:Summary name W2:0/gradient is illegal; using W2_0/gradient instead.
INFO:tensorflow:Summary name W3:0/gradient is illegal; using W3_0/gradient instead.
INFO:tensorflow:Summary name b1:0/gradient is illegal; using b1_0/gradient instead.
INFO:tensorflow:Summary name b2:0/gradient is illegal; using b2_0/gradient instead.
INFO:tensorflow:Summary name b3:0/gradient is illegal; using b3_0/gradient instead.


#### 5. 开始训练

In [6]:
with tf.Session() as sess:
    #初始化
    sess.run(init)
    #在Tensorboard中写入日志
    summary_writer = tf.summary.FileWriter(logs_path,graph = tf.get_default_graph())
    
    #训练循环
    for epoch in range(training_epochs):
        avg_cost = 0
        total_batch = int(mnist.train.num_examples/batch_size)
        #遍历所有的batch
        for i in range(total_batch):
            batch_xs,batch_ys = mnist.train.next_batch(batch_size)
            #运行optimizer，cost和summary节点
            _, c, summary = sess.run([apply_grads,loss,merged_summary_op],
                                    feed_dict = {x:batch_xs,y:batch_ys})
            #每次迭代写入日志
            summary_writer.add_summary(summary,epoch * total_batch)
            #计算平均的loss
            avg_cost += c / total_batch
        #每个epoch显示logs
        if (epoch+1) % display_step == 0:
            print("Epoch:", '%04d' % (epoch+1), "cost=", "{:.9f}".format(avg_cost))
    print('训练结束！')
    
    #测试模型
    #计算accuracy
    print("Accuracy:", acc.eval({x: mnist.test.images, y: mnist.test.labels}))
    print("运行命令行:\n" \
          "--> tensorboard --logdir=/tmp/tensorflow_logs " \
          "\n然后浏览器中打开 http://0.0.0.0:6006/ ")


InvalidArgumentError: Infinity in summary histogram for: b1_0/gradient
	 [[Node: b1_0/gradient = HistogramSummary[T=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:CPU:0"](b1_0/gradient/tag, SGD/gradients/Model/Add_grad/Reshape_1)]]

Caused by op 'b1_0/gradient', defined at:
  File "/usr/local/Cellar/python3/3.6.3/Frameworks/Python.framework/Versions/3.6/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/local/Cellar/python3/3.6.3/Frameworks/Python.framework/Versions/3.6/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/usr/local/Cellar/python3/3.6.3/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/usr/local/Cellar/python3/3.6.3/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/usr/local/Ce